# Workshop Azure Databricks
## 13. MLflow Serving Models with Azure ML service

<img src="https://mlflow.org/docs/0.2.1/_static/MLflow-logo-final-black.png"><br>

# Documentation
Présentation https://azure.microsoft.com/fr-fr/services/databricks/

Documentation Azure Databricks : https://docs.microsoft.com/fr-fr/azure/databricks/

Documentation Azure ML : https://docs.microsoft.com/en-us/azure/machine-learning/

Github : https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/azure-databricks

## 0. Setup

In [0]:
import datetime
now = datetime.datetime.now()
print(now)

2021-02-10 14:06:10.487734

In [0]:
import sys
sys.version

Out[2]: '3.7.6 (default, Jan 8 2020, 19:59:22) \n[GCC 7.3.0]'

## 1. Create or load an Azure ML Workspace

Before models can be deployed to Azure ML, you must create or obtain an Azure ML Workspace. The `azureml.core.Workspace.create()` function will load a workspace of a specified name or create one if it does not already exist. For more information about creating an Azure ML Workspace, see the [Azure ML Workspace management documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-workspace).

In [0]:
import azureml
from azureml.core import Workspace

subscription_id = "70b8f39e-8863-49f7-b6ba-34a80799550c" #Votre ID Azure ML service
resource_group = "AMLworkshop-rg" # Le ressource groupe Azure ML service
workspace_name = "AMLworkshop" # Le nom Azure ML service
workspace_region = "westeurope" # région Azure ML service

In [0]:
workspace = Workspace.create(name = workspace_name,
                             location = workspace_region,
                             resource_group = resource_group,
                             subscription_id = subscription_id,
                             exist_ok=True)

## 2. Build an Azure Container Image for model deployment

### 2.1 Use MLflow to build a Container Image for the trained model

Use the `mlflow.azuereml.build_image` function to build an Azure Container Image for the trained MLflow model. This function also registers the MLflow model with a specified Azure ML workspace. The resulting image can be deployed to Azure Container Instances (ACI) or Azure Kubernetes Service (AKS) for real-time serving.

> Choisir le runID du modèle à utiliser depuis l'icone **EXPERIMENT (en haut à droite)**

In [0]:
# Récupération de l'ID du modèle depuis MLflow
run_id1 = "07682b4558594b57b48c4ad1402f72a4"
model_uri = "runs:/" + run_id1 + "/model"

### Génération image

In [0]:
import mlflow.azureml

model_image, azure_model = mlflow.azureml.build_image(model_uri=model_uri, 
                                                      workspace=workspace,
                                                      model_name="diabetes",
                                                      image_name="diabetesimageadb",
                                                      description="Sklearn ElasticNet for Diabetes prediction with Azure Databricks",
                                                      synchronous=False)

/databricks/python/lib/python3.7/site-packages/jwt/api_jwt.py:81: DeprecationWarning: It is strongly recommended that you pass in a value for the "algorithms" argument when calling decode(). This argument will be mandatory in a future version.
 DeprecationWarning
Registering model diabetes
2021/02/10 14:06:24 INFO mlflow.azureml: Registered an Azure Model with name: `diabetes` and version: `7`
/databricks/python/lib/python3.7/site-packages/mlflow/azureml/__init__.py:222: DeprecationWarning: ContainerImage class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 tags=tags,
/databricks/python/lib/python3.7/site-packages/azureml/core/image/container.py:161: DeprecationWarning: ContainerImageConfig class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 base_image, base_image_registry, cuda_version=cuda_version)
/databricks/python/lib/python3.7/site-packages/mlflow/azureml/__init__.py:228: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 models=[registered_model],
/databricks/python/lib/python3.7/site-packages/jwt/api_jwt.py:81: DeprecationWarning: It is strongly recommended that you pass in a value for the "algorithms" argument when calling decode(). This argument will be mandatory in a future version.
 DeprecationWarning
Creating image
/databricks/python/lib/python3.7/site-packages/azureml/core/image/image.py:407: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 image = Image(workspace, id=image_id)
2021/02/10 14:06:27 INFO mlflow.azureml: Building an Azure Container Image with name: `diabetesimageadb` and version: `3`

In [0]:
model_image.wait_for_creation(show_output=True)

Running...........................................................................................
Succeeded
Image creation operation finished for image diabetesimageadb:3, operation "Succeeded"

> Prévoir 8 minutes de temps de traitement.

## 3. Deploy the model to "dev" using Azure Container Instances (ACI)

### 3.1 Create an ACI webservice deployment using the model's Container Image

Using the Azure ML SDK, deploy the Container Image for the trained MLflow model to ACI.

In [0]:
from azureml.core.webservice import AciWebservice, Webservice

dev_webservice_name = "diabete-mlmodel-adb-aci"

dev_webservice_deployment_config = AciWebservice.deploy_configuration()
dev_webservice = Webservice.deploy_from_image(name=dev_webservice_name, image=model_image, deployment_config=dev_webservice_deployment_config, workspace=workspace)

/local_disk0/tmp/1612965189152-0/PythonShell.py:6: DeprecationWarning: deploy_from_image has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 # [ML-9441] We keep this launch script minimal because all classes and methods defined here are
/databricks/python/lib/python3.7/site-packages/jwt/api_jwt.py:81: DeprecationWarning: It is strongly recommended that you pass in a value for the "algorithms" argument when calling decode(). This argument will be mandatory in a future version.
 DeprecationWarning
/databricks/python/lib/python3.7/site-packages/azureml/core/image/image.py:821: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 image = cls(None)

#### Déploiement ACI (Prévoir 4 minutes environ)

In [0]:
dev_webservice.wait_for_deployment()

ACI service creation operation finished, operation "Succeeded"

> Le modèle déployé est visible dans Azure ML Studio section **Endpoints**

## 4. Query the ACI deployed model

### 4.1 Load diabetes dataset

In [0]:
from sklearn import datasets
diabetes = datasets.load_diabetes()

### 4.2 Create sample input vector

In [0]:
import pandas as pd
import numpy as np

X = diabetes.data
y = diabetes.target
Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
cols = ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6', 'progression']
data = pd.DataFrame(d, columns=cols)
sample = data.drop(["progression"], axis=1).iloc[[0]]
                                                 
query_input = sample.to_json(orient='split')
query_input = eval(query_input)
query_input.pop('index', None)

Out[11]: [0]

### 4.3 Evaluate the sample input vector by sending an HTTP request
Query the ACI webservice's scoring endpoint by sending an HTTP POST request that contains the input vector.

In [0]:
import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=json.dumps(inputs), headers=headers)
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

Scoring URL :

In [0]:
print('This is scoring endpoint of the deployed model:', dev_webservice.scoring_uri)

This is scoring endpoint of the deployed model: http://2807aa80-7b7f-4012-b926-a231f62d0677.westeurope.azurecontainer.io/score

Appel du modèle :

In [0]:
dev_prediction = query_endpoint_example(scoring_uri=dev_webservice.scoring_uri, inputs=query_input)

Sending batch prediction request with inputs: {'columns': ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'], 'data': [[0.0380759064, 0.0506801187, 0.0616962065, 0.021872355, -0.0442234984, -0.0348207628, -0.0434008457, -0.002592262, 0.0199084209, -0.0176461252]]}
Received response: [201.3209729614444]

## 5. Deploy the model to production using [Azure Kubernetes Service (AKS)](https://azure.microsoft.com/en-us/services/kubernetes-service/). <br> Option 1 or Option 2.

### Option 1: Create a new AKS cluster

If you do not have an active AKS cluster for model deployment, create one using the Azure ML SDK.

> ##### Prévoir 6 minutes de temps de traitement

In [0]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this)
prov_config = AksCompute.provisioning_configuration()

aks_cluster_name = "aks-azuredb" 
# Create the cluster
aks_target = ComputeTarget.create(workspace = workspace, 
                                  name = aks_cluster_name, 
                                  provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

/databricks/python/lib/python3.7/site-packages/jwt/api_jwt.py:81: DeprecationWarning: It is strongly recommended that you pass in a value for the "algorithms" argument when calling decode(). This argument will be mandatory in a future version.
 DeprecationWarning
Creating..........................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None

> Vous pouvez visualiser le compute inference AKS ainsi créé dans Azure ML Studio section **Compute et Inference Clusters**

### Option 2: Connect to an existing AKS cluster

If you already have an active AKS cluster running, you can add it to your Workspace using the Azure ML SDK.

In [0]:
#from azureml.core.compute import AksCompute, ComputeTarget

# Get the resource group from https://porta..azure.com -> Find your resource group
#resource_group = "<resource-group>"

# Give the cluster a local name
#aks_cluster_name = "diabetes-cluster"

# Attatch the cluster to your workgroup
#attach_config = AksCompute.attach_configuration(resource_group=resource_group, cluster_name=aks_cluster_name)
#aks_target = ComputeTarget.attach(workspace, name="diabetes-compute", attach_config)

# Wait for the operation to complete
#aks_target.wait_for_completion(True)
#print(aks_target.provisioning_state)
#print(aks_target.provisioning_errors)

### 6. Deploy to the model's image to the specified AKS cluster

In [0]:
from azureml.core.webservice import Webservice, AksWebservice

prod_webservice_name = "diabete-mlmodel-adb-aks"
prod_webservice_deployment_config = AksWebservice.deploy_configuration()

prod_webservice = Webservice.deploy_from_image(workspace = workspace, 
                                               name = prod_webservice_name,
                                               image = model_image,
                                               deployment_config = prod_webservice_deployment_config,
                                               deployment_target = aks_target)

/local_disk0/tmp/1612965189152-0/PythonShell.py:10: DeprecationWarning: deploy_from_image has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 import sys
/databricks/python/lib/python3.7/site-packages/jwt/api_jwt.py:81: DeprecationWarning: It is strongly recommended that you pass in a value for the "algorithms" argument when calling decode(). This argument will be mandatory in a future version.
 DeprecationWarning
/databricks/python/lib/python3.7/site-packages/azureml/core/image/image.py:821: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 image = cls(None)

#### Déploiement

In [0]:
# Prévoir 2 minutes de temps de traitement
prod_webservice.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running....................
Succeeded
AKS service creation operation finished, operation "Succeeded"

> Le modèle déployé est visible dans Azure ML Studio

## 7. Query the deployed model in production

#### Evaluate the sample input vector by sending an HTTP request
Query the AKS webservice's scoring endpoint by sending an HTTP POST request that includes the input vector. The production AKS deployment may require an authorization token (service key) for queries. Include this key in the HTTP request header.

In [0]:
prod_scoring_uri = prod_webservice.scoring_uri
prod_service_key = prod_webservice.get_keys()[0] if len(prod_webservice.get_keys()) > 0 else None

In [0]:
prod_scoring_uri

Out[19]: 'http://51.105.168.134:80/api/v1/service/diabete-mlmodel-adb-aks/score'

In [0]:
import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=json.dumps(inputs), headers=headers)
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

### Test URL :

In [0]:
prod_prediction1 = query_endpoint_example(scoring_uri=prod_scoring_uri, service_key=prod_service_key, inputs=query_input)

Sending batch prediction request with inputs: {'columns': ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'], 'data': [[0.0380759064, 0.0506801187, 0.0616962065, 0.021872355, -0.0442234984, -0.0348207628, -0.0434008457, -0.002592262, 0.0199084209, -0.0176461252]]}
Received response: [201.3209729614444]

In [0]:
dev_webservice.delete() # Suppression instance ACI
prod_webservice.delete() # Suppression instance AKS
aks_target.delete() # Suppression instance AKS

> End of labs